In [ ]:
import tarfile
import os

def extract_tar_gz_files(tar_files, extract_to_dir):
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)

    for tar_file in tar_files:
        with tarfile.open(tar_file, 'r:gz') as tar_ref:
            tar_ref.extractall(extract_to_dir)

tar_files = ['/content/drive/MyDrive/CycleGAN_Facial_Aging/part1.tar.gz', '/content/drive/MyDrive/CycleGAN_Facial_Aging/part2.tar.gz', '/content/drive/MyDrive/CycleGAN_Facial_Aging/part3.tar.gz']
extract_to_dir = 'extracted_data'
extract_tar_gz_files(tar_files, extract_to_dir)


In [ ]:
import shutil

def combine_extracted_data(source_dirs, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for source_dir in source_dirs:
        for filename in os.listdir(source_dir):
            if filename.endswith('.jpg'):
                src_path = os.path.join(source_dir, filename)
                dst_path = os.path.join(target_dir, filename)
                shutil.copy(src_path, dst_path)

source_dirs = ['extracted_data/part1', 'extracted_data/part2', 'extracted_data/part3']
target_dir = 'combined_dataset'
combine_extracted_data(source_dirs, target_dir)


In [ ]:
!pip install tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow_model_optimization as tfmot

# Custom generator to load images and extract age from filenames
class UTKFaceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_filenames, batch_size, img_size=(128, 128), shuffle=True):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
        self.img_size = img_size
        self.indexes = np.arange(len(self.image_filenames))
        self.shuffle = shuffle
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_filenames = [self.image_filenames[k] for k in batch_indexes]
        X, y = self.__data_generation(batch_filenames)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_filenames):
        X = np.empty((len(batch_filenames), *self.img_size, 3))
        y = np.empty((len(batch_filenames),))

        for i, filename in enumerate(batch_filenames):
            parts = filename.split('_')
            age = int(parts[0])

            img_path = os.path.join(data_dir, filename)
            img = Image.open(img_path).convert('RGB')
            img = img.resize(self.img_size)
            img_array = np.array(img) / 255.0  # Normalize image

            X[i] = img_array
            y[i] = (age - age_min) / (age_max - age_min)  # Normalize age

        return X, y

# Data directory and image filenames
data_dir = '/content/combined_dataset'
image_filenames = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Extract age range to normalize later
ages = [int(f.split('_')[0]) for f in image_filenames]
age_min = min(ages)
age_max = max(ages)

# Split data into training and validation sets
train_filenames, test_filenames = train_test_split(image_filenames, test_size=0.2, random_state=42)

# Create data generators for training and validation
batch_size = 32
train_generator = UTKFaceDataGenerator(train_filenames, batch_size, img_size=(128, 128), shuffle=True)
test_generator = UTKFaceDataGenerator(test_filenames, batch_size, img_size=(128, 128), shuffle=False)

# Model definition with reduced complexity
def build_age_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),  # Reduced filters
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),  # Reduced dense layer size
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='linear')  # Age prediction (regression)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Pruning parameters
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.90,
                                                             begin_step=0,
                                                             end_step=1000)
}

# Build and prune the model
pruned_model = prune_low_magnitude(build_age_model((128, 128, 3)), **pruning_params)
pruned_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Pruning callbacks
callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

# Train the pruned model
pruned_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    callbacks=callbacks
)

# Strip pruning and save the pruned model
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)
model_for_export.save('pruned_age_model.h5')

# Convert the model to TensorFlow Lite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the smaller TFLite model
with open('age_prediction_model.tflite', 'wb') as f:
    f.write(tflite_model)


Epoch 1/20
603/603 [==============================] - 310s 482ms/step - loss: 0.0878 - mae: 0.1607 - val_loss: 0.0290 - val_mae: 0.1278
Epoch 2/20
603/603 [==============================] - 249s 413ms/step - loss: 0.0327 - mae: 0.1391 - val_loss: 0.0282 - val_mae: 0.1261
Epoch 3/20
603/603 [==============================] - 246s 407ms/step - loss: 0.0291 - mae: 0.1300 - val_loss: 0.0278 - val_mae: 0.1241
Epoch 4/20
603/603 [==============================] - 282s 468ms/step - loss: 0.0278 - mae: 0.1268 - val_loss: 0.0272 - val_mae: 0.1233
Epoch 5/20
603/603 [==============================] - 248s 411ms/step - loss: 0.0272 - mae: 0.1252 - val_loss: 0.0262 - val_mae: 0.1231
Epoch 6/20
603/603 [==============================] - 277s 459ms/step - loss: 0.0269 - mae: 0.1242 - val_loss: 0.0262 - val_mae: 0.1251
Epoch 7/20
603/603 [==============================] - 244s 405ms/step - loss: 0.0268 - mae: 0.1239 - val_loss: 0.0260 - val_mae: 0.1217
Epoch 8/20
603/603 [============================

/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
